# Quantum Circuit Simulation and Analysis

This notebook demonstrates:
1. Creating quantum circuits programmatically
2. Uploading circuits to S3
3. Triggering Lambda simulations
4. Querying results from DynamoDB
5. Visualizing quantum states and measurements
6. Comparing quantum algorithms

**Prerequisites:**
- AWS account configured (S3, Lambda, DynamoDB)
- Environment variables set (AWS_S3_BUCKET, AWS_DYNAMODB_TABLE)
- Lambda function deployed

## Setup and Imports

In [ ]:
import os
import json
import time
from datetime import datetime
from typing import Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal

import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## Configuration

In [ ]:
# AWS Configuration
BUCKET_NAME = os.environ.get('AWS_S3_BUCKET', 'quantum-circuits-xxxx')
DYNAMODB_TABLE = os.environ.get('AWS_DYNAMODB_TABLE', 'QuantumResults')
LAMBDA_FUNCTION = os.environ.get('AWS_LAMBDA_FUNCTION', 'simulate-quantum-circuit')
AWS_REGION = os.environ.get('AWS_REGION', 'us-east-1')

print(f"Configuration:")
print(f"  S3 Bucket: {BUCKET_NAME}")
print(f"  DynamoDB Table: {DYNAMODB_TABLE}")
print(f"  Lambda Function: {LAMBDA_FUNCTION}")
print(f"  Region: {AWS_REGION}")

# Initialize AWS clients
s3_client = boto3.client('s3', region_name=AWS_REGION)
lambda_client = boto3.client('lambda', region_name=AWS_REGION)
dynamodb = boto3.resource('dynamodb', region_name=AWS_REGION)
table = dynamodb.Table(DYNAMODB_TABLE)

print("\n✅ AWS clients initialized")

## 1. Create Sample Quantum Circuits

Let's create various quantum circuits to demonstrate different algorithms.

In [ ]:
def create_bell_state() -> str:
    """Create Bell state circuit (maximal entanglement)."""
    return """OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q -> c;
"""

def create_ghz_state(n_qubits: int = 4) -> str:
    """Create GHZ state circuit."""
    circuit = f"""OPENQASM 2.0;
include "qelib1.inc";
qreg q[{n_qubits}];
creg c[{n_qubits}];
h q[0];
"""
    for i in range(1, n_qubits):
        circuit += f"cx q[0],q[{i}];\n"
    circuit += "measure q -> c;\n"
    return circuit

def create_superposition(n_qubits: int = 3) -> str:
    """Create equal superposition of all basis states."""
    circuit = f"""OPENQASM 2.0;
include "qelib1.inc";
qreg q[{n_qubits}];
creg c[{n_qubits}];
"""
    for i in range(n_qubits):
        circuit += f"h q[{i}];\n"
    circuit += "measure q -> c;\n"
    return circuit

# Create circuits
circuits = {
    'bell_state': create_bell_state(),
    'ghz_3qubit': create_ghz_state(3),
    'ghz_5qubit': create_ghz_state(5),
    'superposition_3q': create_superposition(3),
}

print("Created quantum circuits:")
for name in circuits.keys():
    print(f"  - {name}")

## 2. Upload Circuits to S3

In [ ]:
def upload_circuit(circuit_name: str, circuit_code: str, algorithm_type: str = 'custom'):
    """Upload circuit to S3."""
    s3_key = f"circuits/{algorithm_type}/{circuit_name}.qasm"
    
    try:
        s3_client.put_object(
            Bucket=BUCKET_NAME,
            Key=s3_key,
            Body=circuit_code,
            ContentType='text/plain'
        )
        print(f"✅ Uploaded: {s3_key}")
        return s3_key
    except ClientError as e:
        print(f"❌ Error uploading {circuit_name}: {e}")
        return None

# Upload all circuits
uploaded_keys = {}
for name, code in circuits.items():
    algo_type = 'bell' if 'bell' in name else 'ghz' if 'ghz' in name else 'basic'
    s3_key = upload_circuit(name, code, algo_type)
    if s3_key:
        uploaded_keys[name] = s3_key

## 3. Trigger Lambda Simulations

In [ ]:
def trigger_lambda_simulation(s3_key: str) -> dict:
    """Trigger Lambda function to simulate circuit."""
    event = {
        'Records': [{
            's3': {
                'bucket': {'name': BUCKET_NAME},
                'object': {'key': s3_key}
            }
        }]
    }
    
    try:
        response = lambda_client.invoke(
            FunctionName=LAMBDA_FUNCTION,
            InvocationType='RequestResponse',
            Payload=json.dumps(event)
        )
        
        result = json.loads(response['Payload'].read())
        return result
    except ClientError as e:
        print(f"❌ Error invoking Lambda: {e}")
        return None

# Trigger simulations
print("Triggering Lambda simulations...\n")
simulation_results = {}

for name, s3_key in uploaded_keys.items():
    print(f"Simulating: {name}")
    result = trigger_lambda_simulation(s3_key)
    
    if result:
        status = result.get('statusCode', 'unknown')
        if status == 200:
            print(f"  ✅ Success")
            simulation_results[name] = result
        else:
            print(f"  ❌ Failed: {result.get('body', 'Unknown error')}")
    
    time.sleep(1)  # Brief delay between invocations

print(f"\n✅ Completed {len(simulation_results)} simulations")

## 4. Query Results from DynamoDB

In [ ]:
def decimal_to_float(obj):
    """Convert DynamoDB Decimal to float."""
    if isinstance(obj, Decimal):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: decimal_to_float(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [decimal_to_float(item) for item in obj]
    return obj

# Wait for DynamoDB writes to complete
print("Waiting for results to be written to DynamoDB...")
time.sleep(5)

# Query all results
response = table.scan()
items = response.get('Items', [])

# Handle pagination
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    items.extend(response.get('Items', []))

# Convert Decimal to float
items = [decimal_to_float(item) for item in items]

print(f"✅ Retrieved {len(items)} results from DynamoDB")

# Convert to DataFrame
if items:
    df = pd.DataFrame(items)
    print("\nResults Summary:")
    display(df[['CircuitID', 'AlgorithmType', 'NumQubits', 'NumGates', 
                'Fidelity', 'Entanglement', 'ExecutionTimeMs']].head(10))
else:
    print("⚠️  No results found in DynamoDB")

## 5. Visualize Measurement Probabilities

In [ ]:
def plot_measurement_probabilities(circuit_id: str, probs: dict):
    """Plot measurement probability distribution."""
    if not probs:
        print(f"No measurement data for {circuit_id}")
        return
    
    # Sort by state
    states = sorted(probs.keys())
    probabilities = [probs[state] for state in states]
    
    # Create bar plot
    plt.figure(figsize=(12, 5))
    bars = plt.bar(range(len(states)), probabilities, color='steelblue', alpha=0.8)
    
    # Highlight highest probability states
    max_prob = max(probabilities)
    for i, (state, prob) in enumerate(zip(states, probabilities)):
        if prob > max_prob * 0.8:
            bars[i].set_color('coral')
    
    plt.xticks(range(len(states)), [f'|{s}⟩' for s in states], rotation=45)
    plt.ylabel('Probability')
    plt.xlabel('Basis State')
    plt.title(f'Measurement Probabilities: {circuit_id}')
    plt.ylim(0, 1.0)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot for each circuit
if items:
    for item in items[:3]:  # Show first 3
        circuit_id = item.get('CircuitID', 'Unknown')
        probs = item.get('MeasurementProbabilities', {})
        plot_measurement_probabilities(circuit_id, probs)

## 6. Compare Algorithms

In [ ]:
if items and len(items) > 0:
    # Group by algorithm type
    algo_groups = {}
    for item in items:
        algo = item.get('AlgorithmType', 'unknown')
        if algo not in algo_groups:
            algo_groups[algo] = []
        algo_groups[algo].append(item)
    
    # Create comparison plots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Qubits vs Execution Time
    for algo, group_items in algo_groups.items():
        qubits = [item['NumQubits'] for item in group_items]
        times = [item['ExecutionTimeMs'] for item in group_items]
        axes[0, 0].scatter(qubits, times, label=algo, s=100, alpha=0.6)
    axes[0, 0].set_xlabel('Number of Qubits')
    axes[0, 0].set_ylabel('Execution Time (ms)')
    axes[0, 0].set_title('Execution Time vs Qubits')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # 2. Entanglement by Algorithm
    algo_names = list(algo_groups.keys())
    entanglements = [np.mean([item['Entanglement'] for item in algo_groups[algo]]) 
                     for algo in algo_names]
    axes[0, 1].bar(algo_names, entanglements, color='teal', alpha=0.7)
    axes[0, 1].set_xlabel('Algorithm Type')
    axes[0, 1].set_ylabel('Average Entanglement Entropy')
    axes[0, 1].set_title('Entanglement by Algorithm')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # 3. Fidelity Distribution
    all_fidelities = [item['Fidelity'] for item in items]
    axes[1, 0].hist(all_fidelities, bins=20, color='mediumseagreen', alpha=0.7, edgecolor='black')
    axes[1, 0].set_xlabel('Fidelity')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].set_title('Fidelity Distribution')
    axes[1, 0].axvline(np.mean(all_fidelities), color='red', linestyle='--', 
                       label=f'Mean: {np.mean(all_fidelities):.3f}')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # 4. Gate Count vs Qubits
    qubits_all = [item['NumQubits'] for item in items]
    gates_all = [item['NumGates'] for item in items]
    axes[1, 1].scatter(qubits_all, gates_all, c=all_fidelities, 
                       cmap='viridis', s=100, alpha=0.6)
    axes[1, 1].set_xlabel('Number of Qubits')
    axes[1, 1].set_ylabel('Number of Gates')
    axes[1, 1].set_title('Gate Count vs Qubits (colored by fidelity)')
    axes[1, 1].grid(alpha=0.3)
    plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1], label='Fidelity')
    
    plt.tight_layout()
    plt.show()
else:
    print("Not enough data for comparison plots")

## 7. Detailed Analysis: Bell State vs GHZ State

In [ ]:
# Compare Bell and GHZ states
bell_items = [item for item in items if item.get('AlgorithmType') == 'bell']
ghz_items = [item for item in items if item.get('AlgorithmType') == 'ghz']

if bell_items and ghz_items:
    print("Bell State Analysis:")
    print(f"  Average entanglement: {np.mean([item['Entanglement'] for item in bell_items]):.3f}")
    print(f"  Average execution time: {np.mean([item['ExecutionTimeMs'] for item in bell_items]):.1f} ms")
    
    print("\nGHZ State Analysis:")
    print(f"  Average entanglement: {np.mean([item['Entanglement'] for item in ghz_items]):.3f}")
    print(f"  Average execution time: {np.mean([item['ExecutionTimeMs'] for item in ghz_items]):.1f} ms")
    
    # Entanglement scaling with qubits (GHZ)
    if len(ghz_items) > 1:
        plt.figure(figsize=(10, 5))
        ghz_qubits = [item['NumQubits'] for item in ghz_items]
        ghz_entanglement = [item['Entanglement'] for item in ghz_items]
        
        plt.plot(ghz_qubits, ghz_entanglement, 'o-', markersize=10, linewidth=2)
        plt.xlabel('Number of Qubits')
        plt.ylabel('Entanglement Entropy')
        plt.title('GHZ State: Entanglement Scaling')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

## 8. Summary Statistics

In [ ]:
if items:
    print("="*70)
    print("Overall Statistics")
    print("="*70)
    print(f"Total circuits simulated: {len(items)}")
    print(f"Average qubits: {np.mean([item['NumQubits'] for item in items]):.1f}")
    print(f"Average gates: {np.mean([item['NumGates'] for item in items]):.1f}")
    print(f"Average fidelity: {np.mean([item['Fidelity'] for item in items]):.3f}")
    print(f"Average entanglement: {np.mean([item['Entanglement'] for item in items]):.3f}")
    print(f"Average execution time: {np.mean([item['ExecutionTimeMs'] for item in items]):.1f} ms")
    
    print("\nAlgorithm Breakdown:")
    for algo in sorted(algo_groups.keys()):
        count = len(algo_groups[algo])
        print(f"  {algo}: {count} circuits")

## 9. Export Results

In [ ]:
# Export to CSV
if items:
    output_file = 'quantum_results.csv'
    df.to_csv(output_file, index=False)
    print(f"✅ Results exported to: {output_file}")
    
    # Export summary statistics
    summary_file = 'quantum_summary.json'
    summary = {
        'total_circuits': len(items),
        'avg_qubits': float(np.mean([item['NumQubits'] for item in items])),
        'avg_fidelity': float(np.mean([item['Fidelity'] for item in items])),
        'avg_entanglement': float(np.mean([item['Entanglement'] for item in items])),
        'avg_execution_time_ms': float(np.mean([item['ExecutionTimeMs'] for item in items])),
        'algorithm_counts': {algo: len(group_items) for algo, group_items in algo_groups.items()},
        'timestamp': datetime.now().isoformat()
    }
    
    with open(summary_file, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"✅ Summary exported to: {summary_file}")

## Next Steps

1. **Explore More Algorithms**: Create circuits for Grover's algorithm, VQE, quantum error correction
2. **Parameter Sweeps**: Test how circuit parameters affect fidelity and entanglement
3. **Noise Simulation**: Add noise models to simulate realistic quantum hardware
4. **Optimization**: Implement circuit optimization techniques
5. **Scaling Analysis**: Test with larger circuits (up to 10 qubits)
6. **AWS Cleanup**: Remember to delete resources when done to avoid charges

See `cleanup_guide.md` for resource deletion instructions.